# [Workshop 5. Assessment of Survey Data Quality](https://wapor.org/events/annual-conference/current-conference/training-workshops/)

## Part II: Design effect and weighting

Author: <a href="mailto:alexander.seymer@plus.ac.at?subject=Regarding the WAPOR 2023 workshop">Alexander Seymer @ PLUS</a>

Date: September 19, 2023

### Abstract

In this session of the workshop, the design effect, as measure for the impact of the sampling design on the variance of an estimator, and weighting as tool to account for complex sampling designs will be discussed.

### Preparing the R session


In [5]:
source("install.R")

### Weighting

Although we will use R, alternative software packages provide means to employ weights:

| Software | Reference |
| -------- | --------- |
| Python | Paczkowski, W. R. (2022). Modern survey analysis: Using Python for deeper insights. Springer Nature. [https://link.springer.com/book/10.1007/978-3-030-76267-4](https://link.springer.com/book/10.1007/978-3-030-76267-4) | 
| SAS | Lewis, T. H. (2016). [Complex survey data analysis with SAS. CRC Press.](https://www.routledge.com/Complex-Survey-Data-Analysis-with-SAS/Lewis/p/book/9781498776776) |
| SPSS | Zou, D., Lloyd, J. E. V., & Baumbusch, J. L. (2019). Using SPSS to Analyze Complex Survey Data: A Primer. Journal of Modern Applied Statistical Methods, 18. [http://jmasm.com/index.php/jmasm/article/view/1026](http://jmasm.com/index.php/jmasm/article/view/1026) | 
| STATA | Weighing Data in Stata—Stata Help—Reed College. (2014). Retrieved September 7th, 2023, from [https://www.reed.edu/psychology/stata/gs/tutorials/weights.html](https://www.reed.edu/psychology/stata/gs/tutorials/weights.html) |


Complex sampling designs for surveys are commonly applied by employing:

- stratification (divide the population in homogenous groups and sample from each group a specific number);
- clustering (divide the population in groups, e.g. regions, and sample from a random subset of this groups);
- unequal sampling (oversampling of subgroups of interest).

Considering these samples as random samples will result in biased standard errors. 

A commmon approach to account for the bias is providing design weights. Most surveys using complex sampling strategies are providing weights to adjust for the deviation from the random sampling.


### Design effect

And as with all biases, the first and foremost question is how serious is the issue? And of course, the answer is complicated and depends on different factors. Calculating the design effect helps to get an understanding of the gravity of the issue. The 
